# Generate object(sign) and annotations on draw

In [135]:
%matplotlib inline
from matplotlib import pyplot as plt
import os, sys
from glob import glob
from matplotlib import pyplot

from PIL import Image
from PIL import EpsImagePlugin
import math

from PIL import ImageFilter
import numpy as np

from lxml import etree
import xml.etree.ElementTree as xml
import random
import cv2

from sklearn.cluster import KMeans

# preprocessing (CAD Img)

In [ ]:
im = cv2.imread('./data/dataset/1-f_pan.png')

ret1, im1 = cv2.threshold(im,100,255, cv2.THRESH_BINARY_INV)
ret2, im2 = cv2.threshold(im,150,255, cv2.THRESH_BINARY_INV)
ret3, im3 = cv2.threshold(im,175,255, cv2.THRESH_BINARY_INV)
ret4, im4 = cv2.threshold(im,200,255, cv2.THRESH_BINARY_INV)
ret5, im5 = cv2.threshold(im,250,255, cv2.THRESH_BINARY_INV)

cv2.imwrite('./data/test1.png', im1)
cv2.imwrite('./data/test2.png', im2)
cv2.imwrite('./data/test3.png', im3)
cv2.imwrite('./data/test4.png', im4)
cv2.imwrite('./data/test5.png', im5)

In [97]:
im = cv2.imread('./data/dataset/1-f_pan.png')
ret, imGray = cv2.threshold(im,150,255, cv2.THRESH_BINARY_INV)

gray = cv2.cvtColor(imGray, cv2.COLOR_BGR2GRAY)
cv2.imwrite('./data/GrayTest.png', gray)

True

In [100]:
im = Image.open('./data/dataset/1-f_pan.png')
    
data = np.array(im)

r1, g1, b1 = 0,0,0  # Original value
r2, g2, b2 = 255,255,255 # Value that we want to replace it with

red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
mask = (red == r1) & (green == g1) & (blue == b1)
data[:,:,:3][mask] = [r2, g2, b2]

im = Image.fromarray(data)    
imDir = './data/TTEST.png'
im.save(imDir)

In [98]:
im = cv2.imread('./data/dataset/1-f_pan.png',cv2.IMREAD_GRAYSCALE)
cv2.imwrite('./data/Gray.png', im)

ret, imGray = cv2.threshold(im,150,255, cv2.THRESH_BINARY_INV)
cv2.imwrite('./data/testGray.png', imGray)

True

In [57]:
im = Image.open('./data/TTEST.png')
    
data = np.array(im)


r1, g1, b1 = 255,255,128  # Original value
r2, g2, b2 = 255,0,0 # Value that we want to replace it with

red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
mask = (red == r1) & (green == g1) & (blue == b1)
data[:,:,:3][mask] = [r2, g2, b2]

im = Image.fromarray(data)    
imDir = './data/TTEST22.png'
im.save(imDir)

# Sign

In [72]:
s_img =  cv2.imread('./DT_150.png')


blur1 = cv2.blur(s_img, (30,30))
blur2 = cv2.GaussianBlur(s_img, (17,17),0)
blur3 = cv2.medianBlur(s_img, 11)

cv2.imwrite('./TTEST_SIGN1.png',  blur1) 
cv2.imwrite('./TTEST_SIGN2.png',  blur2) 
cv2.imwrite('./TTEST_SIGN3.png',  blur3) 

True

In [103]:
img =  cv2.imread('./DT_150.png')


kernel = np.ones((30, 30), np.uint8)
result1 = cv2.erode(img, kernel, iterations = 1)


kernel = np.ones((50, 50), np.uint8)
result2 = cv2.erode(img, kernel, iterations = 1)

kernel = np.ones((70, 70), np.uint8)
result3 = cv2.erode(img, kernel, iterations = 1)



cv2.imwrite('./TTEST_erode30.png',  result1) 
cv2.imwrite('./TTEST_erode50.png',  result2) 
cv2.imwrite('./TTEST_erode70.png',  result3) 

True

In [108]:
img =  cv2.imread('./TTEST_erode50.png')

kernel = np.ones((15, 15), np.uint8)
result = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)


cv2.imwrite('./TTEST_erode50_dliate.png',  result ) 




True

In [105]:
img =  cv2.imread('./DT_150.png')

kernel = np.ones((7, 7), np.uint8)
result = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)


cv2.imwrite('./TTEST_dilate.png',  result ) 

True

In [ ]:
l_img =  cv2.imread('./data/TTEST.png')
s_img =  cv2.imread('./DT_150.png',-1)  
    
alpha_s = s_img[:, :, 3] / 25
alpha_l = 1.0 - alpha_s
    

imName = ChangeColor(signDir,r2,g2,b2)
s_img =  cv2.imread(imName,-1)
          
h,w,d = l_img.shape

for c in range(0, 3):
    l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[y1:y2, x1:x2, c])

    cv2.imwrite('./data/TTEST_SIGN.png', l_img)      

# preprocessing (Print Img)

In [19]:
im = cv2.imread('./data/n_1f.png')
ret, im = cv2.threshold(im,200,255, cv2.THRESH_BINARY)
cv2.imwrite('./data/Wt_THGray.png', im)


True

In [2]:
draw_data = glob('./data/backGround/draw/*.png')
annotaions_data = glob('./data/backGround/annotations/*.xml')
sign_data = glob('./data/backGround/sign/*.png')

# Reshape Sign 
size = 20, 20

for s in sign_data:
    im = Image.open(s)
    im.thumbnail(size, Image.ANTIALIAS)
    im = im.convert("RGBA")
    datas = im.getdata()
    newData = []
    for item in datas:
        if item[0] > 200 and item[1] > 200 and item[2] > 200:
            newData.append((item[0], item[1], item[2], 0))
        else:
            newData.append(item)

    im.putdata(newData)
    im.save(s)

----------------------------------

# Change color of sign

In [162]:
def ChangeColor(inputImg,r2,g2,b2):
    
    im = Image.open(inputImg)
    im = im.filter(ImageFilter.MaxFilter(size = 3)) 
    
    im = np.array(im)
    ret, im = cv2.threshold(im, 247, 255, cv2.THRESH_TOZERO)
    data = im
    
    
    imName = inputImg.split('sign/')[1]
    imName = imName.split('.png')[0]    

    r1, g1, b1 = 0, 0, 0 # Original value
    r2, g2, b2 = r2,g2,b2 # Value that we want to replace it with

    red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
    mask = (red == r1) & (green == g1) & (blue == b1)
    data[:,:,:3][mask] = [r2, g2, b2]

    im = Image.fromarray(data)    
    imDir = './data/backGround/New/sign/'+imName+'_r'+str(r2)+'g'+str(g2)+'b'+str(b2)+'.png'
    im.save(imDir)
    
    return imDir

# Make n-Sign on Draw with loaction(x,y) information 

In [160]:
def genData(clss, amount, dr_fileName,r2,g2,b2):
    
    l_img =  cv2.imread(dr_fileName)
    s_img =  cv2.imread('./data/backGround/sign/'+clss+'.png',-1)  
    
    signDir = './data/backGround/sign/'+clss+'.png'
    
    alpha_s = s_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    
    dr_fileName = dr_fileName.split('draw/')[1]
    dr_fileName = dr_fileName.split('.png')[0]    
    
    new_dirFileName = './data/draw/dataset/GN_'+dr_fileName+str(random.random())+'.png'          
    new_fileName  = new_dirFileName .split('dataset/')[1]

    imName = ChangeColor(signDir,r2,g2,b2)
    s_img =  cv2.imread(imName,-1)
          
    h,w,d = l_img.shape
      
    # make annotations file
    root = etree.Element("annotation")

    folder = etree.SubElement(root, "folder").text ='draw'  
    filename =  etree.SubElement(root, "filename").text = new_fileName    
    path =  etree.SubElement(root, "path").text = new_dirFileName

    source =  etree.SubElement(root, "source")
    database =  etree.SubElement(source, "database").text= 'Unknown'     
    
    size =  etree.SubElement(root, "size")
    width =  etree.SubElement(size, "width").text = str(w)
    height =  etree.SubElement(size, "height").text = str(h)
    depth =  etree.SubElement(size, "depth").text = str(d)

    segmented =  etree.SubElement(root, "segmented").text = '0'
    
       
    # generate signs and annotaion files.                               
    xmin = random.sample(range(100, w), amount)
    ymin = random.sample(range(100, h), amount)

    location = list(zip(xmin,ymin))
        
        
    for i in range(len(location)):   
            
        x1, y1  = location[i]
        x2 = x1 + s_img.shape[1]
        y2 = y1 + s_img.shape[0]
               
        print(x1, x2, y1, y2)
        
        obj =  etree.SubElement(root, "object")
        name =   etree.SubElement(obj, "name").text = clss
        poset =   etree.SubElement(obj, "pose").text = 'Unspecified'
        truncated =   etree.SubElement(obj, "truncated").text = '0'
        difficult =   etree.SubElement(obj, "difficult").text = '0'

        bndbox = etree.SubElement(obj, "bndbox")
        xmin =  etree.SubElement(bndbox, "xmin").text = str(x1)
        ymin =  etree.SubElement(bndbox, "ymin").text = str(y1)
        xmax =  etree.SubElement(bndbox, "xmax").text = str(x2)
        ymax =  etree.SubElement(bndbox, "ymax").text = str(y2)
        
        for c in range(0, 3):
            l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[y1:y2, x1:x2, c])

    cv2.imwrite(new_fileName, l_img)        
    tree = etree.ElementTree(root)
    
    with open('./data/backGround/'+new_fileName+'.xml', "bw") as fh:
        tree.write(fh)
               

In [6]:
genData('DT_150',10, './data/backGround/draw/b1.png',255,255,72)

1350 1370 635 646
1071 1091 288 299
267 287 161 172
689 709 129 140
1570 1590 910 921
900 920 435 446
1104 1124 778 789
1270 1290 195 206
343 363 709 720
453 473 510 521


----------------------------------

# 보완하는 중 ..

In [ ]:
# Reshape Sign 

size = 20, 20

for s in sign_data:
    im = Image.open(s)
    im.thumbnail(size, Image.ANTIALIAS)
    im = im.convert("RGBA")
    datas = im.getdata()
    newData = []
    for item in datas:
        if item[0] > 200 and item[1] > 200 and item[2] > 200:
            newData.append((item[0], item[1], item[2], 0))
        else:
            newData.append(item)

    im.putdata(newData)
    im.save(s)

In [120]:
sign_data = glob('./data/backGround/sign/*.png')
sign_data 

['./data/backGround/sign/DT_002.png',
 './data/backGround/sign/DT_001.png',
 './data/backGround/sign/SM_142.png',
 './data/backGround/sign/DT_143.png',
 './data/backGround/sign/SM_126.png',
 './data/backGround/sign/DT_150.png',
 './data/backGround/sign/MD_019.png',
 './data/backGround/sign/DT_131.png',
 './data/backGround/sign/DT_127.png',
 './data/backGround/sign/DT_177.png']

In [164]:
draw_data = glob('./data/backGround/draw/*.png')
sign_data = glob('./data/backGround/sign/*.png')
draw_data = glob('./data/backGround/draw/*.png')

for i in draw_data:
    im = cv2.imread(i)
    ret, imGray = cv2.threshold(im,150,255, cv2.THRESH_BINARY_INV)

    gray = cv2.cvtColor(imGray, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(i, gray)

In [173]:
dr_fileName = './data/backGround/draw/10f.png'

l_img =  cv2.imread(dr_fileName)
h,w,d = l_img.shape
    
sign_data = glob('./data/backGround/sign/*.png')

signList=[]
# set a color RGB
r2,g2,b2 = 255, 0, 0

# set an amoumnt of sign

DT_002_amount = 5
DT_001_amount = 5
SM_142_amount = 5
DT_143_amount = 5
SM_126_amount = 5
DT_150_amount = 5
MD_019_amount = 5
DT_131_amount = 5
DT_127_amount = 5
DT_177_amount = 5

signAmount = [DT_002_amount,DT_001_amount, SM_142_amount, DT_143_amount, SM_126_amount, DT_150_amount, MD_019_amount, DT_131_amount, DT_127_amount, DT_177_amount]
total = sum(signAmount)

new_fileName  =  'GEN_'+str(int(random.random()*10))+'.png'
new_dirFileName = './data/backGround/New/draw/'+ new_fileName

root = etree.Element("annotation")

folder = etree.SubElement(root, "folder").text ='draw'  
filename =  etree.SubElement(root, "filename").text = new_fileName    
path =  etree.SubElement(root, "path").text = new_dirFileName

source =  etree.SubElement(root, "source")
database =  etree.SubElement(source, "database").text= 'Unknown'     
    
size =  etree.SubElement(root, "size")
width =  etree.SubElement(size, "width").text = str(w)
height =  etree.SubElement(size, "height").text = str(h)
depth =  etree.SubElement(size, "depth").text = str(d)

segmented =  etree.SubElement(root, "segmented").text = '0'

print(total)
print(new_dirFileName)

for i, j  in zip(sign_data , signAmount):
    signDir = i
    clssName = singDir.split('sign/')[1]
    clssName = singDir.split('.png')[0]
    signList.append(clssName)

    imDir = ChangeColor(signDir,r2,g2,b2)
  
    s_img =  cv2.imread(imDir,-1)
    alpha_s = s_img[:, :, 3] / 255.0
    alpha_l = 1.0 - Salpha 
        
    xmin = random.sample(range(50, w), total)
    ymin = random.sample(range(50, h), total)
    
    location = list(zip(xmin,ymin))
        
        
    for i in range(j):   
            
        x1, y1  = location[i]
        x2 = x1 + s_img.shape[1]
        y2 = y1 + s_img.shape[0]
               
        print(x1, x2, y1, y2)
        
        obj =  etree.SubElement(root, "object")
        name =   etree.SubElement(obj, "name").text = clssName
        poset =   etree.SubElement(obj, "pose").text = 'Unspecified'
        truncated =   etree.SubElement(obj, "truncated").text = '0'
        difficult =   etree.SubElement(obj, "difficult").text = '0'

        bndbox = etree.SubElement(obj, "bndbox")
        xmin =  etree.SubElement(bndbox, "xmin").text = str(x1)
        ymin =  etree.SubElement(bndbox, "ymin").text = str(y1)
        xmax =  etree.SubElement(bndbox, "xmax").text = str(x2)
        ymax =  etree.SubElement(bndbox, "ymax").text = str(y2)
        
        for c in range(0, 3):
            l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[y1:y2, x1:x2, c])
            
cv2.imwrite(new_dirFileName, l_img)        
tree = etree.ElementTree(root)
    
with open(new_dirFileName +'.xml', "bw") as fh:
    tree.write(fh)
    
    
  

50
./data/backGround/New/draw/GEN_1.png
1472 1492 67 87
1023 1043 388 408
303 323 224 244
890 910 645 665
452 472 594 614
623 643 469 489
1210 1230 254 274
693 713 632 652
910 930 855 875
1433 1453 118 138
660 680 818 833


ValueError: operands could not be broadcast together with shapes (20,20) (15,20) 